## making the datawarehouse 

In [1]:
import pandas as pd
import sqlite3
import numpy as np
conn = sqlite3.connect('CourseDataWarehouse.db')
c = conn.cursor()

In [2]:
% load_ext sql

In [3]:
%%sql

sqlite:///CourseDataWarehouse.db

'Connected: @CourseDataWarehouse.db'

In [4]:
%%sql

DROP TABLE IF EXISTS LOCATION_W;

CREATE TABLE LOCATION_W(
    LID INTEGER PRIMARY KEY,
    BUILDING TEXT,
    ROOM TEXT
);

DROP TABLE IF EXISTS FACULTY_W;

CREATE TABLE FACULTY_W(
    FID INTEGER PRIMARY KEY,
    LNAME TEXT,
    FNAME TEXT
);

DROP TABLE IF EXISTS COURSE_W;

CREATE TABLE COURSE_W(
    CID INTEGER PRIMARY KEY,
    CRN TEXT,
    CREDITS TEXT,
    PREREQS TEXT,
    COREQS TEXT,
    ATTRIBUTES TEXT,
    DESCRIPTION TEXT,
    PROGRAM_NAME TEXT,
    PROGRAM_CODE TEXT
);

DROP TABLE IF EXISTS TERM_W;

CREATE TABLE TERM_W(
    TERMID INTEGER PRIMARY KEY,
    SEMESTER TEXT,
    YEAR TEXT,
    TERM TEXT
);


 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

## populating the dimension tables

In [5]:
%%sql

--popluting the location dimension table--
INSERT INTO LOCATION_W(BUILDING, ROOM)
SELECT DISTINCT BUILDING, ROOM 
FROM Location;

 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) no such table: Location [SQL: '--popluting the location dimension table--\nINSERT INTO LOCATION_W(BUILDING, ROOM)\nSELECT DISTINCT BUILDING, ROOM \nFROM Location;'] (Background on this error at: http://sqlalche.me/e/e3q8)


In [6]:
%%sql

SELECT * FROM LOCATION_W
LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


LID,BUILDING,ROOM


In [7]:
%%sql

--populating the faculty_w dimension table--
INSERT INTO FACULTY_W(LNAME, FNAME)
SELECT DISTINCT LNAME, FNAME
FROM FACULTY;

 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) no such table: FACULTY [SQL: '--populating the faculty_w dimension table--\nINSERT INTO FACULTY_W(LNAME, FNAME)\nSELECT DISTINCT LNAME, FNAME\nFROM FACULTY;'] (Background on this error at: http://sqlalche.me/e/e3q8)


In [8]:
%%sql

SELECT * FROM FACULTY_W
LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


FID,LNAME,FNAME


In [9]:
%%sql

--populating course_w dimension table
INSERT INTO COURSE_W(CRN,CREDITS,PREREQS, COREQS,ATTRIBUTES,DESCRIPTION,PROGRAM_NAME,PROGRAM_CODE)
SELECT DISTINCT CRN, CREDITS,PREREQS, COREQS,ATTRIBUTES,DESCRIPTION,PROGRAMNAME,PROGRAMCODE
FROM COURSE
    LEFT JOIN PROGRAMS USING (PID)
    LEFT JOIN COURSE_OFFERING USING (CID);
    


 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) no such table: COURSE [SQL: '--populating course_w dimension table\nINSERT INTO COURSE_W(CRN,CREDITS,PREREQS, COREQS,ATTRIBUTES,DESCRIPTION,PROGRAM_NAME,PROGRAM_CODE)\nSELECT DISTINCT CRN, CREDITS,PREREQS, COREQS,ATTRIBUTES,DESCRIPTION,PROGRAMNAME,PROGRAMCODE\nFROM COURSE\n    LEFT JOIN PROGRAMS USING (PID)\n    LEFT JOIN COURSE_OFFERING USING (CID);'] (Background on this error at: http://sqlalche.me/e/e3q8)


In [10]:
%%sql

SELECT * FROM COURSE_W LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


CID,CRN,CREDITS,PREREQS,COREQS,ATTRIBUTES,DESCRIPTION,PROGRAM_NAME,PROGRAM_CODE


In [11]:
%%sql

--populating the term_w table--
INSERT INTO TERM_W(SEMESTER, YEAR, TERM)
SELECT DISTINCT SEMESTER, YEAR, TERM
FROM COURSE_OFFERING;

 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) no such table: COURSE_OFFERING [SQL: '--populating the term_w table--\nINSERT INTO TERM_W(SEMESTER, YEAR, TERM)\nSELECT DISTINCT SEMESTER, YEAR, TERM\nFROM COURSE_OFFERING;'] (Background on this error at: http://sqlalche.me/e/e3q8)


In [12]:
%%sql

SELECT * FROM TERM_W LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


TERMID,SEMESTER,YEAR,TERM


## creating the fact table.

In [13]:
%%sql

DROP TABLE IF EXISTS COURSE_OFFERING_W;

CREATE TABLE COURSE_OFFERING_W(
    COID INTEGER PRIMARY KEY,
    ACT INTEGER,
    CAP INTEGER, 
    REM INTEGER,
    AVG_CAP INTEGER,
    MAX_CAP INTEGER,
    AVG_CLASS_BUILDING INTERGER,
    NUM_ROOM_BUILDING INTERGER,
    TERMID INTEGER,
    CID INTEGER,
    LID INTEGER,
    FID INTEGER,
    FOREIGN KEY (TERMID) REFERENCES TERM_W (TERMID),
    FOREIGN KEY (CID) REFERENCES COURSE_W (CID),
    FOREIGN KEY (LID) REFERENCES LOCATION_W (LID),
    FOREIGN KEY (FID) REFERENCES FACULTY_W (FID)
);

 * sqlite:///CourseDataWarehouse.db
Done.
Done.


[]

In [14]:
%%sql

--populating the course_offering_w table
INSERT INTO COURSE_OFFERING_W (ACT, CAP, REM, FID, LID, TERMID, CID)
SELECT DISTINCT ACT, CAP, REM, FID, LID,TERMID, CID
FROM COURSE_OFFERING
    LEFT JOIN FACULTY_W USING (FID)
    LEFT JOIN CLASS_MEETINGS USING (COID)
    LEFT JOIN LOCATION USING (LID)
    JOIN TERM_W USING (SEMESTER)
    JOIN COURSE_W USING (CID)
;

 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) no such table: COURSE_OFFERING [SQL: '--populating the course_offering_w table\nINSERT INTO COURSE_OFFERING_W (ACT, CAP, REM, FID, LID, TERMID, CID)\nSELECT DISTINCT ACT, CAP, REM, FID, LID,TERMID, CID\nFROM COURSE_OFFERING\n    LEFT JOIN FACULTY_W USING (FID)\n    LEFT JOIN CLASS_MEETINGS USING (COID)\n    LEFT JOIN LOCATION USING (LID)\n    JOIN TERM_W USING (SEMESTER)\n    JOIN COURSE_W USING (CID)\n;'] (Background on this error at: http://sqlalche.me/e/e3q8)


In [15]:
%%sql

SELECT * FROM COURSE_OFFERING_W LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


COID,ACT,CAP,REM,AVG_CAP,MAX_CAP,AVG_CLASS_BUILDING,NUM_ROOM_BUILDING,TERMID,CID,LID,FID


## dropping unnecessary tables

In [16]:
%%sql

DROP TABLE COURSE;
DROP TABLE PROGRAMS;
DROP TABLE COURSE_OFFERING;
DROP TABLE CLASS_MEETINGS;
DROP TABLE FACULTY;
DROP TABLE LOCATION;

 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) no such table: COURSE [SQL: 'DROP TABLE COURSE;'] (Background on this error at: http://sqlalche.me/e/e3q8)
